N.B. This is a notebook I created for my talk at [Hopperx1 London 2019](https://londontechweek.com/event/hopperx1-london-people-and-skills) organised by fabulous people at [AnitaB.org](https://anitab.org/).

It is best served as an interactive HTML slide deck using [nbconvert](https://nbconvert.readthedocs.io/en/latest/install.html), as otherwise the styling of most HTML elements will be completely off!

Here's how to do it. From your terminal, run:

```
$ jupyter nbconvert presentation/hopperx1.ipynb --to slides --post serve
```

It should open up a browser automatically.

Modify the path to this notebook depending on where you're lauching it from.

In [1]:
###########
# Imports #
###########

#####################
# Utility functions #
#####################

<div class='title'>
    <h1 id='main-title'>Seeing is believing</h1>

    <h3 id='subtitle'>Neural nets, what do they "see"?</h3>

    <hr>

    <p>by Misa Ogura</p>
</div>

<head>
    <link rel="stylesheet" href="https://use.fontawesome.com/releases/v5.8.2/css/all.css" integrity="sha384-oS3vJWv+0UjzBfQzYUhtDYW+Pj2yciDJxpsK1OYPAYjqT085Qq/1cq5FLXAZQ7Ay" crossorigin="anonymous">
<head>

<h1>Hello, I'm Misa 👋</h1>

<hr>

<div class='profile-container'>
    <img id='profile-image' src='images/misa_ogura_profile.jpg' width='200' height='200'>

    <ul id='profile-list'>
        <li>Originally from Tokyo, now based in London</li>
        <li>Cancer Cell Biologist, turned Software Engineer</li>
        <li>Currently at BBC R&D</li>
        <li>Co-founder of Women Driven Development</li>
        <li>Women in Data Science London Ambassador</li>
    </ul>
</div>


<div class='social-container'>
    <a href='https://github.com/misaogura' target='_blank'><i class="fab fa-github"></i></a>
    <a href='https://medium.com/@misaogura' target='_blank'><i class="fab fa-medium-m"></i></a>
    <a href='https://twitter.com/misa_ogura' target='_blank'><i class="fab fa-twitter"></i></a>
    <a href='https://www.linkedin.com/in/misaogura/' target='_blank'><i class="fab fa-linkedin-in"></i></a>
    <a href='https://womendrivendev.org' target='_blank'>
        <img id='wdd-logo' src='images/wdd-logo.png' width='20' height='20'>
    </a>
</div>

## Convolutional Neural Network (CNN)

---

### Image Convolution

---

### CNN Architecture

---

## Visualisation Techniques

---

## Demo

---